In [1]:
%matplotlib inline
import os
import copy

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from torchvision import datasets
from torchvision import transforms

from tqdm import tqdm

In [2]:
transform = transforms.Compose([transforms.ToTensor()])
batchsize = 20000

TrainSet    = datasets.MNIST(root="../downloads/MNIST", train=True, download=True, transform=transform)
TrainLoader = torch.utils.data.DataLoader(TrainSet, batch_size=batchsize, shuffle=True, num_workers=0)
TestSet     = datasets.MNIST(root="../downloads/MNIST", train=False, download=True, transform=transform)
TestLoader  = torch.utils.data.DataLoader(TrainSet, batch_size=batchsize, shuffle=True, num_workers=0)

classes    = ('zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine')

In [3]:
use_gpu = torch.cuda.is_available()

if(use_gpu):
    device = "cuda"
    print("GPU is available!")
else:
    device = "cpu"
    print("GPU is not available :(")


GPU is available!


---
## Defining the LeNet

In [21]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(1, 6, kernel_size=5), nn.MaxPool2d(kernel_size=2, stride=2))
        self.conv2 = nn.Sequential(nn.Conv2d(6, 16, kernel_size=5), nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc1 = nn.Sequential(nn.Linear(16*4*4, 120))
        self.fc2 = nn.Sequential(nn.Linear(120, 84))
        self.fc3 = nn.Sequential(nn.Linear(84, 10))

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        x = x.view(-1, 16*4*4)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x))
        
        return x

In [22]:
net = LeNet()

if(use_gpu):
    net = net.cuda()

In [23]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4)

In [24]:
iterations = 50

for epoch in tqdm(range(iterations)):
    runningLoss = 0
    net.train(True)

    for i, data in enumerate(TrainLoader):
        inputs, labels = data
        
        if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labesl = Variable(inputs), Variable(labels)
        
        optimizer.zero_grad()

        outputs = net(inputs)
        
        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        runningLoss += loss.item()
    
    if(epoch %10 == 0):
        print("Loss : %f"%(runningLoss/60000))

100%|██████████| 50/50 [03:07<00:00,  3.75s/it]


In [44]:
TestData = list(TrainLoader)
indices = np.random.uniform(0, TestData[0][0].shape[0], size=10)

for i in indices:    
    test_img, test_label = TestData[0][0][int(i)], TestData[0][1][int(i)]

    out = net((test_img.double().to(device)))
    
    if device == "cuda":
        output_img = output_img.cpu()

    test_img = test_img.view(-1, 28, 28)

    img = np.array(test_img.numpy())[0]
    plt.title("Input image")
    plt.imshow(img, cmap="gray")
    plt.show()

    print("Output : %d, Prediction : %d"%(np.argmax(out), test_label))

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [6, 1, 5, 5], but got 3-dimensional input of size [1, 28, 28] instead